In [1]:
import numpy as np
import torch
import pandas as pd

In [2]:
# Add decord to read videos and not images
from decord import VideoReader
from decord import cpu, gpu
import decord

decord.bridge.set_bridge('torch')

In [3]:
video_path = "/data/imeza/ActivityNet/test_videos/v_te1znGPS_Vw.mp4"

vr = VideoReader(video_path, ctx=cpu(0))

In [4]:
def _get_test_indices(start_frame, end_frame, new_length, num_segments):
    tick = (end_frame - new_length + 1) / float(num_segments)
    offsets = np.array([int(tick / 2.0 + tick * x) + start_frame for x in range(num_segments)])
    return offsets

In [5]:
def _get_all_indices_list(total_frames, num_frames):

    idx_list = list()
    list_frames = range(0, total_frames, num_frames*num_frames)
    sum_indices = _get_test_indices(0, num_frames*num_frames, 1, 8)
    
    for idx, frames in enumerate(list_frames): 
        
        if idx == 0:
            sampled_frames = sum_indices + frames
            idx_list.append(sampled_frames)
        else:
            sampled_frames = sum_indices + frames
            if sum(total_frames < sampled_frames) > 0:
                pass
            else:
                idx_list.append(sampled_frames)
    
    return idx_list

In [6]:
test = {"a":1}
test["a"]

1

In [7]:
def _cache_batch(frames_list):
    
    count = 0
    i = 0
    output_list = list()
    aux_list = list()
    idx_frame_list = list()
    for idx, batch in enumerate(frames_list):
        
        count += len(batch)
        aux_list.extend(batch)
        idx_frame_list.append(idx)
        
        if count == 400:
            output_list.append(
                (i,
                 np.array(aux_list), 
                 np.array(idx_frame_list)
                )
            )
            count = 0
            aux_list = list()
            idx_frame_list = list()
            i += 1
    
    if count < 400 and len(aux_list) != 0:
        output_list.append(
            (i, 
             np.array(aux_list), 
             np.array(idx_frame_list)
            )
        )

    return output_list

In [8]:
idx_list   = _get_all_indices_list(len(vr), 8)
cache_list = _cache_batch(idx_list)

In [16]:
loaded_frames = None
cache_loaded  = None

In [52]:
q_frame = 52

In [26]:
# check and load frames (esto debería estar en un getitem)
if loaded_frames == None:
    for idx, cache in enumerate(cache_list):
        if q_frame in cache[2]:
            print(f"Loading frames by idx: {idx}")
            # estos dos deberían ser selfs
            loaded_frames = vr.get_batch(cache[1])
            cache_loaded  = cache
            break
        else:
            print("Out of range")
elif q_frame in cache_loaded[2]:
    print(f"Using cache")
elif q_frame not in cache_loaded[2]:
    

In [53]:
cache_loaded

array([[0., 0.],
       [0., 0.],
       [0., 0.]])

In [57]:
list_a = [1, 2, 3, 4]
list_a.remove(4)
list_a

[1, 2, 3]

In [51]:
idx = 0
cache_loaded = np.zeros((3,2))
while 
    if q_frame in cache_list[idx][2]:
        print(f"Loading frames by idx: {idx}")
        loaded_frames = vr.get_batch(cache_list[idx][1])
        cache_loaded  = cache_list[idx]
    idx += 1

In [22]:
# transformar esta parte de arriba en diccionario
min_index = int(np.where(cache_loaded[1] == idx_list[q_frame].min())[0])
max_index = int(np.where(cache_loaded[1] == idx_list[q_frame].max())[0])+1

In [23]:
loaded_frames[min_index: max_index].shape

torch.Size([8, 720, 1280, 3])